# Ensemble learning for image classification (CIFAR10)

### Summary

Test

Group number and member names:

In [ ]:
GROUP = "36"
NAME1 = "Timothy Hellberg"
NAME2 = "Lars Liberg"

## 0. Imports

In [ ]:
# YOUR CODE HERE
# For dealing with files
import os, sys
import shutil

# For using regex expressions
import re

# For splitting the data
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier

# Packages for defining the architecture of our model
from keras.models import Sequential
from keras.layers import Dense, Flatten, BatchNormalization, Input, Dropout
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras import regularizers
from keras.optimizers import Adam
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.models import load_model 
from keras.wrappers.scikit_learn import KerasClassifier

# For generating data
from keras.preprocessing.image import ImageDataGenerator

# One-hot encoding
from keras.utils import np_utils

# Callbacks for training
from keras.callbacks import TensorBoard, EarlyStopping

# Ploting
import matplotlib.pyplot as plt
%matplotlib inline

# Ndarray computations
import numpy as np

# Confusion matrix for assessment step
from sklearn.metrics import confusion_matrix

# Dataset
from keras.datasets import cifar10

# Other
import pandas as pd
from subprocess import check_output
from IPython.display import display
from IPython.display import Image as _Imgdis
from PIL import Image
from time import time
from time import sleep
from scipy import ndimage
from scipy.misc import imresize
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Model, Input
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Activation, Average, Dropout, Maximum
from keras.utils import to_categorical
from keras.losses import categorical_crossentropy
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.layers import Concatenate
from keras import backend as K 

## 1. Loading the data and preprocessing

In [ ]:
#home_path = "/Users/timhell/Google Drive/11 Färdigheter/Programmering/Python/Deep machine learning/deep-machine-learning/Project"
home_path = "/home/student/deep-machine-learning/Project"
os.chdir(home_path)

# Dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train / 255.
x_test = x_test / 255.

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

y_train = to_categorical(y_train, num_classes=10)
print('x_train shape: {} | y_train shape: {}\nx_test shape : {} | y_test shape : {}'.format(x_train.shape, y_train.shape,                                                                
                                                                                            x_test.shape, y_test.shape))
input_shape = x_train[0,:,:,:].shape
print("input_shape:",input_shape)
model_input = Input(shape=input_shape)

#plt.imshow(x_train[np.random.randint(50000)]);

## 2. Training

### 2.1 Functions, learners and combiners

In [ ]:
# Universal functions

def compile_and_train(model, num_epochs): 
    model.compile(loss=categorical_crossentropy, optimizer=Adam(), metrics=['acc']) 
    filepath = home_path + '/weights/CIFAR10/' + model.name + '.{epoch:02d}-{val_acc:.2f}.hdf5'
    checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=0, save_weights_only=True, save_best_only=True, mode='auto', period=1)
    tensor_board = TensorBoard(log_dir='logs/', histogram_freq=0, batch_size=32)
    history = model.fit(x=x_train, y=y_train, batch_size=256, epochs=num_epochs, verbose=1, callbacks=[checkpoint, tensor_board], validation_split=0.2)
    return history

def compile_and_train_bagging(model, num_epochs, x_train, y_train): 
    indices = np.random.randint(50000, size=50000)
    x_train_bag = x_train[indices,:,:,:]
    y_train_bag = y_train[indices,:]
    model.compile(loss=categorical_crossentropy, optimizer=Adam(), metrics=['acc']) 
    filepath = home_path + '/weights/CIFAR10/' + model.name + '.{epoch:02d}-{val_acc:.2f}.hdf5'
    checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=0, save_weights_only=True, save_best_only=True, mode='auto', period=1)
    tensor_board = TensorBoard(log_dir='logs/', histogram_freq=0, batch_size=32)
    history = model.fit(x=x_train_bag, y=y_train_bag, batch_size=256, epochs=num_epochs, verbose=1, callbacks=[checkpoint, tensor_board], validation_split=0.2)
    del x_train_bag, y_train_bag
    return history

def evaluate_test_accuracy(model):
    pred = model.predict(x_test, batch_size = 256)
    pred = np.argmax(pred, axis=1)
    pred = np.expand_dims(pred, axis=1) # make same shape as y_test
    test_accuracy = np.sum(np.equal(pred, y_test)) / y_test.shape[0]  
    return test_accuracy

In [ ]:
# Custom base learners

def base_cnn_reg(model_input):
    x = Conv2D(128, (5, 5), activation='relu')(model_input)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dense(10, activation='softmax')(x)
    model = Model(model_input, x, name='CIFAR10_base_cnn_reg')
    return model

def base_cnn_2_reg(model_input): # same as the bagged model
    x = Conv2D(128, (3, 3), activation='relu')(model_input)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dense(10, activation='softmax')(x)
    model = Model(model_input, x, name='CIFAR10_base_cnn_2_reg')
    return model

def base_cnn_2_reg_fewer_filters_and_dense_neurons(model_input):
    x = Conv2D(64, (3, 3), activation='relu')(model_input)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Flatten()(x)
    x = Dense(64, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dense(10, activation='softmax')(x)
    model = Model(model_input, x, name='CIFAR10_base_cnn_2_reg_fewer_filters_and_dense_neurons')
    return model

def base_cnn_3_reg(model_input):
    x = Conv2D(128, (3, 3), activation='relu')(model_input)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dense(10, activation='softmax')(x)
    model = Model(model_input, x, name='CIFAR10_base_cnn_3_reg')
    return model

def base_cnn_4_conv_concise(model_input):
    x = Conv2D(128, (3, 3), activation='relu')(model_input)
    x = Conv2D(128, (3, 3), activation='relu')(model_input)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu')(x)
    x = Conv2D(128, (3, 3), activation='relu')(model_input)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dense(10, activation='softmax')(x)
    model = Model(model_input, x, name='CIFAR10_base_cnn_4_conv_concise')
    return model

def base_learner_bagging(model_input, num):
    x = Conv2D(128, (3, 3), activation='relu')(model_input)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dense(10, activation='softmax')(x)
    model = Model(model_input, x, name='CIFAR10_base_learner_' + str(num) + '_bagging')
    return model

In [ ]:
# Custom combiners

def average_ensemble(models, model_input):
    outputs = [model.outputs[0] for model in models]
    #print(np.shape(np.array(outputs)))
    #print(outputs[0])
    y = Average()(outputs)
    model = Model(models[0].input, y, name='CIFAR10_average_ensemble')
    return model

def concatenate_ensemble(models, model_input):
    outputs = [model.outputs[0] for model in models]
    print(np.shape(np.array(outputs)))
    y = Concatenate(axis=-1)(outputs)
    model = Model(model_input, y, name='CIFAR10_concatenate_ensemble')
    return model

def basic_stacking_ensemble(models, model_input, num):
    outputs = [model.outputs[0] for model in models]
    y = Concatenate(axis=-1)(outputs)
    y = Dense(10, activation='softmax',
              kernel_initializer='random_uniform', 
              bias_initializer='random_uniform')(y)
    model = Model(model_input, y, name='CIFAR10_basic_stacking_' + str(num) + '_ensemble')
    return model

def small_stacking_ensemble(models, model_input, num):
    outputs = [model.outputs[0] for model in models]
    y = Concatenate(axis=-1)(outputs)
    y = Dense(20, activation='relu',
              kernel_initializer='random_uniform', 
              bias_initializer='random_uniform')(y)
    y = Dense(10, activation='softmax',
              kernel_initializer='random_uniform', 
              bias_initializer='random_uniform')(y)
    model = Model(model_input, y, name='CIFAR10_small_stacking_' + str(num) + '_ensemble')
    return model

def large_stacking_ensemble(models, model_input, num):
    outputs = [model.outputs[0] for model in models]
    y = Concatenate(axis=-1)(outputs)
    y = Dense(200, activation='relu',
              kernel_initializer='random_uniform', 
              bias_initializer='random_uniform')(y)
    y = Dense(10, activation='softmax',
              kernel_initializer='random_uniform', 
              bias_initializer='random_uniform')(y)
    model = Model(model_input, y, name='CIFAR10_large_stacking_' + str(num) + '_ensemble')
    return model

def deep_stacking_ensemble(models, model_input, num):
    outputs = [model.outputs[0] for model in models]
    y = Concatenate(axis=-1)(outputs)
    y = Dense(20, activation='relu',
              kernel_initializer='random_uniform', 
              bias_initializer='random_uniform')(y)
    y = Dense(20, activation='relu',
              kernel_initializer='random_uniform', 
              bias_initializer='random_uniform')(y)
    y = Dense(10, activation='softmax',
              kernel_initializer='random_uniform', 
              bias_initializer='random_uniform')(y)
    model = Model(model_input, y, name='CIFAR10_deep_stacking_' + str(num) + '_ensemble')
    return model

def bn_deep_stacking_ensemble(models, model_input, num):
    outputs = [model.outputs[0] for model in models]
    y = Concatenate(axis=-1)(outputs)
    y = Dense(20, activation='relu',
              kernel_initializer='random_uniform', 
              bias_initializer='random_uniform')(y)
    y = BatchNormalization()(y)
    y = Dense(20, activation='relu',
              kernel_initializer='random_uniform', 
              bias_initializer='random_uniform')(y)
    y = BatchNormalization()(y)
    y = Dense(20, activation='relu',
              kernel_initializer='random_uniform', 
              bias_initializer='random_uniform')(y)
    y = BatchNormalization()(y)    
    y = Dense(10, activation='softmax',
              kernel_initializer='random_uniform', 
              bias_initializer='random_uniform')(y)
    model = Model(model_input, y, name='CIFAR10_bn_deep_stacking_' + str(num) + '_ensemble')
    return model

def pop_stacking_ensemble(models, model_input, num):
    for model in models:
        model.layers.pop()
    outputs = [model.outputs[0] for model in models]
    y = Concatenate(axis=-1)(outputs)
    y = Dense(20, activation='relu',
              kernel_initializer='random_uniform', 
              bias_initializer='random_uniform')(y)
    y = Dense(10, activation='softmax',
              kernel_initializer='random_uniform', 
              bias_initializer='random_uniform')(y)
    model = Model(model_input, y, name='CIFAR10_pop_stacking_' + str(num) + '_ensemble')
    return model

### 2.2 Train base learners

#### 2.2.1 Bagging (train base learners)

In [ ]:
# Train bagged model
base_model = base_learner_bagging(model_input, 3)
print(base_model.summary())

models = []
for i in np.arange(0,50):
    base_model = base_learner_bagging(model_input, i)
    _ = compile_and_train_bagging(base_model, 10, x_train, y_train)
    models.append(base_model)
    base_model.save_weights('weights_CIFAR10/weights_CIFAR10_base_learner_' + str(i) + '_bagging.hdf5')
    del base_model


In [ ]:
# Train single model with full dataset
base_model_full = base_learner_bagging(model_input, 50)
print(base_model_full.summary())
_ = compile_and_train(base_model_full, 20)

In [ ]:
# Load bagged model (only weights)

models_bag = []
for i in np.arange(0,50):
    base_model = base_learner_bagging(model_input, i)
    base_model.load_weights('weights_CIFAR10/weights_CIFAR10_base_learner_' + str(i) + '_bagging.hdf5')
    models_bag.append(base_model)
    print(i)

In [ ]:
# Freeze weights in bagged model

model_0 = models_bag[2]
model_0.summary()

for model in models_bag:
    for layer in model.layers:
        layer.trainable = False
        
model_0.summary()

In [ ]:
# Evaluate the different bagged models

for model in models_bag:
    print(evaluate_test_accuracy(model))


#### 2.2.2 Non-bagging (train base learners)

In [ ]:
# Train models


base_model = base_cnn_reg(model_input)
_ = compile_and_train(base_model, 20)
base_model.save_weights('weights_CIFAR10/weights_CIFAR10_base_learner_' + base_model.name + '_non_bagging.hdf5')

base_model = base_cnn_2_reg(model_input)
_ = compile_and_train(base_model, 20)
base_model.save_weights('weights_CIFAR10/weights_CIFAR10_base_learner_' + base_model.name + '_non_bagging.hdf5')

base_model = base_cnn_2_reg_fewer_filters_and_dense_neurons(model_input)
_ = compile_and_train(base_model, 20)
base_model.save_weights('weights_CIFAR10/weights_CIFAR10_base_learner_' + base_model.name + '_non_bagging.hdf5')

base_model = base_cnn_3_reg(model_input)
_ = compile_and_train(base_model, 20)
base_model.save_weights('weights_CIFAR10/weights_CIFAR10_base_learner_' + base_model.name + '_non_bagging.hdf5')

base_model = base_cnn_4_conv_concise(model_input)
_ = compile_and_train(base_model, 20)
base_model.save_weights('weights_CIFAR10/weights_CIFAR10_base_learner_' + base_model.name + '_non_bagging.hdf5')

In [ ]:
# Train single model

base_model = base_cnn_reg(model_input)
_ = compile_and_train(base_model, 10)
base_model.save_weights('weights_CIFAR10/weights_CIFAR10_base_learner_' + base_model.name + '_non_bagging.hdf5')

In [ ]:
# Train model without saving weights

base_model = base_cnn(model_input)
print(base_model.summary())

_ = compile_and_train(base_model, 10)

In [ ]:
# Load models (only weights)

models_nonbag = []

base_model = base_cnn_reg(model_input)
base_model.load_weights('weights_CIFAR10/weights_CIFAR10_base_learner_' + base_model.name + '_non_bagging.hdf5')
models_nonbag.append(base_model)

base_model = base_cnn_2_reg(model_input)
base_model.load_weights('weights_CIFAR10/weights_CIFAR10_base_learner_' + base_model.name + '_non_bagging.hdf5')
models_nonbag.append(base_model)

base_model = base_cnn_2_reg_fewer_filters_and_dense_neurons(model_input)
base_model.load_weights('weights_CIFAR10/weights_CIFAR10_base_learner_' + base_model.name + '_non_bagging.hdf5')
models_nonbag.append(base_model)

base_model = base_cnn_3_reg(model_input)
base_model.load_weights('weights_CIFAR10/weights_CIFAR10_base_learner_' + base_model.name + '_non_bagging.hdf5')
models_nonbag.append(base_model)

base_model = base_cnn_4_conv_concise(model_input)
base_model.load_weights('weights_CIFAR10/weights_CIFAR10_base_learner_' + base_model.name + '_non_bagging.hdf5')
models_nonbag.append(base_model)


In [ ]:
# Freeze weights in non-bagged model

model_0 = models_nonbag[0]
model_0.summary()

for model in models_nonbag:
    for layer in model.layers:
        layer.trainable = False
        
model_0.summary()

In [ ]:
# Evaluate the different non-bagged models

for model in models_nonbag:
    print(evaluate_test_accuracy(model))

### 2.3 Evaluate simple combiners

#### 2.3.1 Bagging (evaluate simple combiners)

In [ ]:
# Max voting

max_voting_accuracies = np.zeros((51,1))
for k in np.arange(2,50):
    n_models = k
    models_short = models_bag[0:n_models]

    pred_new = np.zeros((10000,10))
    max_voting_model = concatenate_ensemble(models_short, model_input)
    pred = max_voting_model.predict(x_test, batch_size = 256)
    pred = np.expand_dims(pred, axis=1)
    for i in range(n_models):
        j = i*10
        #print(i)
        #print(np.shape(np.array(pred)))
        #print(np.shape(np.array(pred)))
        pred_part = pred[:,:,j:j+10]
        #print(np.shape(np.array(pred)))
        index = np.argmax(pred_part, axis=2)
        #print(np.shape(np.array(index)))
        index = to_categorical(index, num_classes=10)
        #print(np.shape(np.array(index)))
        #print(index[i,:])
        pred_new += index
        #print(pred_new[0:3,:])

    pred_new = np.expand_dims(pred_new, axis=1)
    pred_new = np.argmax(pred_new, axis=2)
    max_voting_accuracies[k] = np.sum(np.equal(pred_new, y_test)) / y_test.shape[0]
    print(k,"test_acc:", max_voting_accuracies[k])

In [ ]:
#np.savetxt('CIFAR10_max_voting_accuracies.txt', max_voting_accuracies, fmt='%.4f')

In [ ]:
# Average

average_accuracies = np.zeros((51,1))

for i in np.arange(2,50):
    n_models = i
    models_short = models_bag[0:n_models]
    average_model = average_ensemble(models_short, model_input)
    average_accuracies[i] = evaluate_test_accuracy(average_model)
    print(i,"test_acc:", average_accuracies[i])

In [ ]:
#np.savetxt('CIFAR10_average_accuracies.txt', average_accuracies, fmt='%.4f')

#### 2.3.2 Non-bagging (evaluate simple combiners)

In [ ]:
# Max voting

models_short = models_nonbag[0:5]

pred_new = np.zeros((10000,10))
max_voting_model = concatenate_ensemble(models_short, model_input)
pred = max_voting_model.predict(x_test, batch_size = 256)
pred = np.expand_dims(pred, axis=1)
for i in range(len(models_short)):
    j = i*10
    #print(i)
    #print(np.shape(np.array(pred)))
    #print(np.shape(np.array(pred)))
    pred_part = pred[:,:,j:j+10]
    #print(np.shape(np.array(pred)))
    index = np.argmax(pred_part, axis=2)
    #print(np.shape(np.array(index)))
    index = to_categorical(index, num_classes=10)
    #print(np.shape(np.array(index)))
    #print(index[i,:])
    pred_new += index
    #print(pred_new[0:3,:])

pred_new = np.expand_dims(pred_new, axis=1)
pred_new = np.argmax(pred_new, axis=2)
max_voting_accuracy = np.sum(np.equal(pred_new, y_test)) / y_test.shape[0]
print("max voting, test acc:", max_voting_accuracy)

In [ ]:
# Average

models_short = models_nonbag[0:5]

average_model = average_ensemble(models_short, model_input)
average_accuracy = evaluate_test_accuracy(average_model)
print("average, test acc:", average_accuracy)

### 2.4 Train and evaluate stacking combiners

#### 2.4.1 Bagging (train and evaluate stacking combiners)

In [ ]:
# Train ensemble once

n_models = 3

ensemble_model = basic_stacking_ensemble(models_bag[0:n_models], model_input, n_models)
_ = compile_and_train(ensemble_model, num_epochs=1)
accuracy = evaluate_test_accuracy(ensemble_model)
print(n_models, "test_acc:", accuracy)

In [ ]:
# Train ensemble multiple times

basic_stacking_accuracies = np.zeros((51,1))

range_list = np.arange(0,50,5) 
range_list[0] = 2 # 2,5,10,15,20,...,45

for i in range_list:
    n_models = i
    ensemble_model = basic_stacking_ensemble(models_bag[0:n_models], model_input, i)
    _ = compile_and_train(ensemble_model, num_epochs=3)
    basic_stacking_accuracies[i] = evaluate_test_accuracy(ensemble_model)
    print(i,"test_acc:", basic_stacking_accuracies[i])


In [ ]:
#np.savetxt('CIFAR10_basic_stacking_accuracies.txt', basic_stacking_accuracies, fmt='%.4f')

In [ ]:
# Evaluate ensemble

n_models = 15

ensemble_model = basic_stacking_ensemble(models_bag[0:n_models], model_input)
ensemble_model.load_weights('weights/CIFAR10/CIFAR10_basic_stacking_ensemble.01-0.99.hdf5')
print(evaluate_test_accuracy(ensemble_model))

 #### 2.4.2 Non-bagging (train and evaluate stacking combiners)

In [ ]:
# Train ensemble once and evaluate

models_short = models_nonbag[0:5]

ensemble_model = pop_stacking_ensemble(models_short, model_input, 0)
_ = compile_and_train(ensemble_model, num_epochs=20)
accuracy = evaluate_test_accuracy(ensemble_model)
print(len(models_short), "test_acc:", accuracy)

In [ ]:
# Evaluate ensemble

models_short = models_nonbag[0:5]

ensemble_model = pop_stacking_ensemble(models_short, model_input, 0)
print(ensemble_model.summary())
ensemble_model.load_weights('weights/CIFAR10/CIFAR10_pop_stacking_0_ensemble.05-0.77.hdf5')
print(evaluate_test_accuracy(ensemble_model))